In [1]:
%pylab inline
import pandas
import gc
import sys
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [5]:
sys.path += ['/home/nath/Projects/Kaggle/tools/python']

In [11]:
import cooking_tools
from imp import reload
reload(cooking_tools)
from cooking_tools import *

In [7]:
ROOT_DIR = '/home/nath/Projects/challenge-cfm/'

## Load data

In [7]:
train_data = pandas.read_csv(ROOT_DIR + 'data/training_input.csv', usecols=['nb_trade', 'offset', 'ID'])
gc.collect()

0

In [2]:
target = get_target(ROOT_DIR + 'data/challenge_output_data_training_file_prediction_of_trading_activity_within_the_order_book.csv')

In [4]:
(target == 1).sum()

198591

In [5]:
target.shape

(587214,)

## nb_trade

In [247]:
nb_trade = get_data(train_data, 'nb_trade', 0)

In [248]:
scores = compute_signal_accuracy_scores(nb_trade.values, target)
scores.max()

0.6851999441430211

## Left EMA nb_trade

In [249]:
for com in range(12, 30, 2):
    signal = nb_trade.ewm(com=com).mean()
    scores = compute_signal_accuracy_scores(signal.values, target)
    print(scores.max())

0.690491030527
0.690688573501
0.690795859772
0.690787344988
0.690656217325
0.690623861148
0.690596613841
0.690537010357
0.690267943203


## Centered rolling mean nb_trade

In [250]:
for window_size in range(80, 120, 2):
    signal = nb_trade.rolling(window_size, center=True).mean()
    scores = compute_signal_accuracy_scores(signal.values, target)
    print(scores.max())

0.694292029822
0.694259673645
0.694435078183
0.694429969313
0.694521928973
0.69449808758
0.694373771742
0.694484463926
0.694545770367
0.694353336262
0.694360148089
0.694210287902
0.694116625285
0.694041695191
0.693876508394
0.693798172387
0.693774330993
0.693697697943
0.693716430467
0.693738568903


## Right EMA nb_trade

In [84]:
for com in range(12, 30, 2):
    reversed_nb_trade = nb_trade.iloc[::-1]
    signal = reversed_nb_trade.ewm(com=com).mean()
    signal = signal.iloc[::-1]
    scores = compute_signal_accuracy_scores(signal.values, target)
    print(scores.max())

0.69280364569
0.693069306931
0.693164672504
0.693057386234
0.693016515274
0.69293647631
0.69280364569
0.692706577159
0.692519251925


## Left rolling mean nb_trade

In [87]:
for window_size in range(25, 50, 2):
    signal = nb_trade.rolling(window_size, center=False).mean()
    scores = compute_signal_accuracy_scores(signal.values, target)
    print(scores.max())

0.685414516684
0.685741484365
0.68600544265
0.686269400934
0.686441399558
0.686431181818
0.686346033984
0.686397122684
0.686625318879
0.686707060799
0.686693437146
0.686538468088
0.686277915717


## Right rolling mean nb_trade

In [90]:
for window_size in range(25, 50, 2):
    reversed_nb_trade = nb_trade.iloc[::-1]
    signal = reversed_nb_trade.rolling(window_size, center=False).mean()
    signal = signal.iloc[::-1]
    scores = compute_signal_accuracy_scores(signal.values, target)
    print(scores.max())

0.688161385798
0.688471323913
0.688794885681
0.688943042911
0.689040111441
0.688932825171
0.688990725698
0.688842568467
0.688895360124
0.688864706904
0.688919201518
0.688897063081
0.688958369521


## Two-sided EMA nb_trade

In [102]:
for com in range(6, 30, 2):
    signal_left = nb_trade.ewm(com=com).mean()   
    
    reversed_nb_trade = nb_trade.iloc[::-1]
    signal_right = reversed_nb_trade.ewm(com=com).mean()
    signal_right = signal_right.iloc[::-1]
    
    scores = compute_signal_accuracy_scores(signal_left.values + signal_right.values, target)
    
    print(scores.max())

0.6979993665
0.698677143256
0.699104585381
0.699273178092
0.699416226452
0.699426444192
0.699394088016
0.699131832688
0.698956428151
0.698893418754
0.698704390563
0.698522174199


In [251]:
scores = compute_signal_accuracy_scores(two_sided_ema(train_data), target)    
print(scores.max())

0.699474126979


## Mean of two-sided EMA nb_trade

In [113]:
train_data['time'] = get_time(train_data)

In [124]:
df = pandas.concat([get_data(train_data, 'time', 0), two_sided_ema(train_data)], axis=1)

In [129]:
mean_of_two_sided_ema = df.groupby('time').mean().values

In [145]:
signal = mean_of_two_sided_ema[df['time']].ravel()

In [146]:
scores = compute_signal_accuracy_scores(signal, target)
print(scores.max())

0.683745619144


## 3 days rolling mean of two-sided EMA

In [155]:
df['two_sided_ema_day_before'] = df['two_sided_ema'].shift(-DAY).fillna(df['two_sided_ema'])

In [157]:
df['two_sided_ema_next_day'] = df['two_sided_ema'].shift(DAY).fillna(df['two_sided_ema'])

In [160]:
signal = df['two_sided_ema'] + 0.5*(df['two_sided_ema_day_before'] + df['two_sided_ema_next_day'])

In [169]:
for alpha in np.linspace(0,0.5, 10):
    signal = df['two_sided_ema'] + alpha*(df['two_sided_ema_day_before'] + df['two_sided_ema_next_day'])
    scores = compute_signal_accuracy_scores(signal, target)
    print(scores.max())

0.699474126979
0.700162121475
0.700366476276
0.700351149666
0.699991825808
0.699532027506
0.69891555719
0.698346769661
0.697709863866
0.697064443286


In [9]:
for alpha in np.linspace(0,0.5, 10):
    tse = two_sided_ema(train_data)
    signal = tse + alpha*(day_shift(tse, -1) + day_shift(tse, 1))
    scores = compute_signal_accuracy_scores(signal, target)
    print(scores.max())

0.699474126979
0.700162121475
0.700366476276
0.700351149666
0.699991825808
0.699532027506
0.69891555719
0.698346769661
0.697709863866
0.697064443286


## Simple MMP

In [11]:
train_data = None
gc.collect()
train_data = pandas.read_csv(ROOT_DIR + 'data/training_input.csv', usecols=['offset', 'bid_size_1', 'ask_size_1', 'bid_size_2', 'ask_size_2', 'ID'])

In [254]:
bid_size_1 = get_data(train_data, 'bid_size_1', 0)
ask_size_1 = get_data(train_data, 'ask_size_1', 0)
simple_mmp = mmp(bid_size_1, ask_size_1)

In [255]:
scores = compute_signal_accuracy_scores(simple_mmp.abs(), target)
print(scores.max())

0.666310748722


## Two levels MMP

In [262]:
q_bid = get_data(train_data, 'bid_size_1', 0) + get_data(train_data, 'bid_size_2', 0)
q_ask = get_data(train_data, 'ask_size_1', 0) + get_data(train_data, 'ask_size_2', 0)

In [263]:
mmp_2 = mmp(q_bid, q_ask)
scores = compute_signal_accuracy_scores(mmp_2.abs(), target)
print(scores.max())

0.661915417548


## Rolling mean sizes MMP

In [270]:
bid_size_ma = get_data(train_data, 'bid_size_1', 0).rolling(3).mean()
ask_size_ma = get_data(train_data, 'ask_size_1', 0).rolling(3).mean()
mmp = mmp(bid_size_ma, ask_size_ma)

In [271]:
scores = compute_signal_accuracy_scores(mmp.abs(), target)
print(scores.max())

0.661811537191


## Epoch averaged sizes MMP

In [222]:
bid_size_ave = get_epoch_average(train_data, 'bid_size_1')
ask_size_ave = get_epoch_average(train_data, 'ask_size_1')
mmp_ave = mmp(bid_size_ave, ask_size_ave)

In [225]:
scores = compute_signal_accuracy_scores(np.abs(mmp_ave), target)
print(scores.max())

0.661820051974


## Consecutive size diff

In [288]:
signal = np.abs(get_data(train_data, 'bid_size_1', 0).values - get_data(train_data, 'bid_size_1', -1000).shift(1).values)

In [289]:
scores = compute_signal_accuracy_scores(signal, target)
print(scores.max())

0.664645257095


In [294]:
signal = np.abs(get_data(train_data, 'bid_size_1', 0).values - get_data(train_data, 'bid_size_1', -1000).values)

In [295]:
scores = compute_signal_accuracy_scores(signal, target)
print(scores.max())

0.670993879574


In [298]:
for offset in OFFSETS:
    signal = np.abs(get_data(train_data, 'bid_size_1', 0).values - get_data(train_data, 'bid_size_1', offset).values)
    scores = compute_signal_accuracy_scores(signal, target)
    print(offset)
    print(scores.max())

0
0.662237276359
-10
0.665457567429
-20
0.665486517692
-50
0.666642825273
-100
0.668412197257
-200
0.670258202291
-500
0.671813001734
-1000
0.670993879574


In [13]:
for offset in OFFSETS:
    signal = get_epoch_open_close(train_data, 'bid_size_1', offset)
    scores = compute_signal_accuracy_scores(signal, target)
    print(offset)
    print(scores.max())

0
0.662237276359
-10
0.665457567429
-20
0.665486517692
-50
0.666642825273
-100
0.668412197257
-200
0.670258202291
-500
0.671813001734
-1000
0.670993879574


In [299]:
for offset in OFFSETS:
    signal = np.abs(get_data(train_data, 'ask_size_1', 0).values - get_data(train_data, 'ask_size_1', offset).values)
    scores = compute_signal_accuracy_scores(signal, target)
    print(offset)
    print(scores.max())

0
0.662237276359
-10
0.665532497522
-20
0.665844138593
-50
0.667470462217
-100
0.668674452585
-200
0.670103233234
-500
0.671542231623
-1000
0.67080655434


In [300]:
for offset in OFFSETS:
    signal = np.abs(get_data(train_data, 'ask_size_2', 0).values - get_data(train_data, 'ask_size_2', offset).values)
    scores = compute_signal_accuracy_scores(signal, target)
    print(offset)
    print(scores.max())

0
0.662237276359
-10
0.663855085199
-20
0.664686128056
-50
0.666440173429
-100
0.667984755132
-200
0.670222440201
-500
0.672114425065
-1000
0.67185046678


In [301]:
for offset in OFFSETS:
    signal = np.abs(get_data(train_data, 'bid_size_2', 0).values - get_data(train_data, 'bid_size_2', offset).values)
    scores = compute_signal_accuracy_scores(signal, target)
    print(offset)
    print(scores.max())

0
0.662237276359
-10
0.663638809701
-20
0.664531158998
-50
0.666344807855
-100
0.668015408352
-200
0.670215628374
-500
0.672388601089
-1000
0.671663141546


## Absurd signals

In [302]:
for offset in OFFSETS:
    signal = np.abs(get_data(train_data, 'bid_size_1', 0).values - get_data(train_data, 'bid_size_2', offset).values)
    scores = compute_signal_accuracy_scores(signal, target)
    print(offset)
    print(scores.max())

0
0.661808131278
-10
0.661808131278
-20
0.661808131278
-50
0.661809834234
-100
0.661816646061
-200
0.661808131278
-500
0.661808131278
-1000
0.661808131278


In [322]:
scores = compute_signal_accuracy_scores(get_epoch_std(train_data, 'bid_size_1'), target)
print(scores.max())

0.674278882997


In [323]:
scores = compute_signal_accuracy_scores(get_epoch_std(train_data, 'bid_size_2'), target)
print(scores.max())

0.674881729659


In [324]:
scores = compute_signal_accuracy_scores(get_epoch_std(train_data, 'ask_size_1'), target)
print(scores.max())

0.67422098247


In [325]:
scores = compute_signal_accuracy_scores(get_epoch_std(train_data, 'ask_size_2'), target)
print(scores.max())

0.674827235045


In [326]:
scores = compute_signal_accuracy_scores(get_data(train_data, 'bid_size_1', 0).rolling(window=8, center=True).std(), target)
print(scores.max())

0.663213070533


In [16]:
scores = compute_signal_accuracy_scores(get_rolling(train_data, 'bid_size_1', -4, 4).std(), target)
print(scores.max())

0.663213070533


In [337]:
stack = np.vstack([get_data(train_data, 'bid_size_1', offset) for offset in OFFSETS])
high = stack.max(axis=0)
low = stack.min(axis=0)
scores = compute_signal_accuracy_scores(high - low, target)
print(scores.max())

0.674924303576


In [19]:
scores = compute_signal_accuracy_scores(get_epoch_high_low(train_data, 'bid_size_1'), target)
print(scores.max())

0.674924303576


In [338]:
stack = np.vstack([get_data(train_data, 'bid_size_2', offset) for offset in OFFSETS])
high = stack.max(axis=0)
low = stack.min(axis=0)
scores = compute_signal_accuracy_scores(high - low, target)
print(scores.max())

0.675205291427


## Entry-based MMP

In [303]:
train_data = pandas.read_csv(ROOT_DIR + 'data/training_input.csv', usecols=['offset', 'bid_entry_1', 'ask_entry_1', 'ID'])
gc.collect()

489

In [265]:
bid_entry_1 = get_data(train_data, 'bid_entry_1', 0)
ask_entry_1 = get_data(train_data, 'ask_entry_1', 0)
mmp_entry = mmp(bid_entry_1, ask_entry_1)

In [266]:
scores = compute_signal_accuracy_scores(mmp_entry.abs(), target)
print(scores.max())

0.665144223401


## Consecutive entry diff

In [241]:
signal = np.abs(get_data(train_data, 'bid_entry_1', 0).values - get_data(train_data, 'bid_entry_1', -1000).shift(1).values)

In [244]:
scores = compute_signal_accuracy_scores(signal, target)
print(scores.max())

0.66401346017


In [277]:
signal = np.abs(get_data(train_data, 'bid_entry_1', 0).values - get_data(train_data, 'bid_entry_1', 0).shift(1).values)

In [278]:
scores = compute_signal_accuracy_scores(signal, target)
print(scores.max())

0.663277782887


In [273]:
signal = np.abs(get_data(train_data, 'bid_entry_1', 0).values - get_data(train_data, 'bid_entry_1', -1000).shift(-1).values)

In [274]:
scores = compute_signal_accuracy_scores(signal, target)
print(scores.max())

0.662107851652


In [275]:
signal = np.abs(get_data(train_data, 'bid_entry_1', 0).values - get_data(train_data, 'bid_entry_1', 0).shift(-1).values)

In [276]:
scores = compute_signal_accuracy_scores(signal, target)
print(scores.max())

0.662010783122


In [279]:
signal = np.abs(get_data(train_data, 'ask_entry_1', 0).values - get_data(train_data, 'ask_entry_1', -1000).shift(1).values)

In [280]:
scores = compute_signal_accuracy_scores(signal, target)
print(scores.max())

0.66401346017


In [305]:
for offset in OFFSETS:
    signal = np.abs(get_data(train_data, 'bid_entry_1', 0).values - get_data(train_data, 'bid_entry_1', offset).values)
    scores = compute_signal_accuracy_scores(signal, target)
    print(offset)
    print(scores.max())

0
0.662237276359
-10
0.665043748957
-20
0.665551230046
-50
0.666554271526
-100
0.668398573603
-200
0.669965293743
-500
0.671150551588
-1000
0.669440783088


## Epoch std bid_entry

In [318]:
scores = compute_signal_accuracy_scores(get_epoch_std(train_data, 'bid_entry_1'), target)
print(scores.max())

0.673192396639


In [320]:
scores = compute_signal_accuracy_scores(get_epoch_std(train_data, 'ask_entry_1'), target)
print(scores.max())

0.672948873835


## Vol

In [8]:
train_data = None
gc.collect()
train_data = pandas.read_csv(ROOT_DIR + 'data/training_input.csv', usecols=['offset', 'bid_1', 'ask_1', 'ID'])

In [331]:
for window_size in range(4, 20, 2):
    scores = compute_signal_accuracy_scores(get_data(train_data, 'bid_1', 0).rolling(window_size, center=True).std(), target)
    print(scores.max())

0.678132673949
0.681272926054
0.681516448859
0.681862149063
0.681758268706
0.681461954245
0.680814830709
0.680418041804


In [345]:
for window_size in range(4, 20, 2):
    high = get_data(train_data, 'bid_1', 0).rolling(window_size, center=True).max()
    low = get_data(train_data, 'bid_1', 0).rolling(window_size, center=True).min()
    scores = compute_signal_accuracy_scores(high - low, target)
    print(scores.max())

0.678241663176
0.682497351902
0.683011644818
0.684389336766
0.683881855678
0.684740145841
0.684302485976
0.684030012908


In [353]:
stack = np.vstack([get_data(train_data, 'bid_1', offset) for offset in OFFSETS])
high = stack.max(axis=0)
low = stack.min(axis=0)
scores = compute_signal_accuracy_scores(high - low, target)
print(scores.max())

0.673890608875


## Trend

In [361]:
for window_size in range(4, 20, 2):
    signal = np.abs(get_data(train_data, 'bid_1', 0) - get_data(train_data, 'bid_1', 0).rolling(window_size, center=False).mean())
    scores = compute_signal_accuracy_scores(signal, target)
    print(scores.max())

0.671826625387
0.672477154836
0.672451610486
0.671874308174
0.671942426441
0.671714230247
0.670598793625
0.670476180745


In [12]:
clf = BaggingLogisticRegression(n_estimators=3)

In [13]:
bid_1 = get_data(train_data, 'bid_1', 0)
X = np.vstack([bid_1.shift(i).fillna(bid_1) for i in range(5)]).T

In [14]:
from sklearn.model_selection import cross_val_score


(587214, 5)

In [22]:
for window_size in range(4, 20, 2):
    signal = np.abs(get_data(train_data, 'bid_1', 0) - get_rolling(train_data, 'bid_1', -window_size, 0).mean())
    scores = compute_signal_accuracy_scores(signal, target)
    print(scores.max())

0.671826625387
0.672477154836
0.672451610486
0.671874308174
0.671942426441
0.671714230247
0.670598793625
0.670476180745


In [24]:
for window_size in range(1, 10, 1):
    signal = np.abs(get_data(train_data, 'bid_1', 0) - get_rolling(train_data, 'bid_1', 0, window_size).mean())
    scores = compute_signal_accuracy_scores(signal, target)
    print(scores.max())

0.662237276359
0.668417306127
0.668115882796
0.66823679272
0.668401979517
0.668250416373
0.667547095267
0.667310384289
0.667080485138


In [4]:
for window_size in range(4, 20, 2):
    signal = np.abs(get_data(train_data, 'bid_1', 0).shift(-window_size) - get_data(train_data, 'bid_1', 0))
    scores = compute_signal_accuracy_scores(signal, target)
    print(scores.max())

0.677020643241
0.678703164434
0.677994734458
0.678817262531
0.679026726202
0.678241663176
0.678159921255
0.678248475002


In [5]:
for window_size in range(4, 20, 2):
    signal = np.abs(get_data(train_data, 'bid_1', 0).shift(window_size) - get_data(train_data, 'bid_1', 0))
    scores = compute_signal_accuracy_scores(signal, target)
    print(scores.max())

0.677027455067
0.678713382174
0.678008358111
0.678834292098
0.679047161682
0.678265504569
0.678187168562
0.678279128222


In [8]:
for window_size in range(2, 10, 1):
    signal = np.abs(get_data(train_data, 'bid_1', 0).shift(window_size) - get_data(train_data, 'bid_1', 0).shift(-window_size))
    scores = compute_signal_accuracy_scores(signal, target)
    print(scores.max())

0.677024049154
0.678708273304
0.678001546285
0.678825777315
0.679036943942
0.678253583872
0.678173544909
0.678263801612


In [25]:
for window_size in range(2, 10, 1):
    signal = get_open_close(train_data, 'bid_1', -window_size, window_size)
    scores = compute_signal_accuracy_scores(signal, target)
    print(scores.max())

0.677024049154
0.678708273304
0.678001546285
0.678825777315
0.679036943942
0.678253583872
0.678173544909
0.678263801612


In [11]:
signal = np.abs(get_data(train_data, 'bid_1', 0) - get_epoch_mean(train_data, 'bid_1'))
scores = compute_signal_accuracy_scores(signal, target)
print(scores.max())

0.673761184168


## bid_1 - bid_2

In [339]:
train_data = None
gc.collect()
train_data = pandas.read_csv(ROOT_DIR + 'data/training_input.csv', usecols=['offset', 'bid_1', 'bid_2', 'ID'])

In [340]:
diff = train_data['bid_1'] - train_data['bid_2']

In [344]:
diff.max()

1.0